In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.datasets.mnist import load_data
from tqdm import tqdm

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def define_G(z_dim, use_bias=False, print_summary=True):
    kernel_init = 'glorot_normal'

    z_in = Input(shape=(z_dim,))

    net = Dense(units=128, kernel_regularizer=l2(1E-5), use_bias=use_bias)(z_in)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)

    net = Dense(units=7*7*32, kernel_regularizer=l2(1E-5), use_bias=use_bias)(net)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)
    net = Reshape((7, 7, 32))(net)

    net = Conv2DTranspose(filters=16, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                          kernel_regularizer=l2(1E-5), padding='same', kernel_initializer=kernel_init)(net)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)

    out = Conv2DTranspose(filters=1, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                          padding='same', kernel_initializer=kernel_init, activation='tanh')(net)

    model = tf.keras.models.Model(inputs=z_in, outputs=out, name='Generator')

    if print_summary:
        print(model.summary())

    return model

def define_C(x_shape, n_classes, use_bias=True, print_summary=True):
    kernel_init = 'glorot_normal'

    x_in = Input(shape=x_shape)

    net = Conv2D(filters=16, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                   kernel_regularizer=l2(1E-5), padding='same', kernel_initializer=kernel_init)(x_in)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)

    net = Conv2D(filters=32, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                   kernel_regularizer=l2(1E-5), padding='same', kernel_initializer=kernel_init)(net)
    net = BatchNormalization()(net)
    net = Flatten()(net)

    net = Dense(units=128, use_bias=use_bias)(net)
    out = Dense(units=n_classes, activation='softmax', use_bias=use_bias)(net)

    model = tf.keras.models.Model(inputs=x_in, outputs=out, name='Classifier')

    if print_summary:
        print(model.summary())

    return model

@tf.function
def train_step(x, y):
    z = tf.random.uniform(minval=-1, maxval=1, shape=(BATCH_SIZE, z_dim))
    # 추가
#     u = tf.ones([BATCH_SIZE,10])/10    
    # 추가
    with tf.GradientTape() as C_tape, tf.GradientTape() as G_tape:
        Gz = generator(z, training=True)

        C_real = classifier(x, training=True)
        C_fake = classifier(Gz, training=True)

        C_real_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, C_real))
        # 추가
#         kl = tf.keras.losses.KLDivergence()
#         C_fake_loss = kl(u, C_fake) * (-1) # uniform 안되도록 kl(u, C_fake)는 uniform 될수록 작아져
#         C_loss = C_real_loss + C_fake_loss

        C_fake_loss = tf.reduce_mean((-1) * C_fake * tf.math.log(C_fake+1e-6))
        C_loss = C_real_loss - C_fake_loss

    C_grad = C_tape.gradient(C_loss, classifier.trainable_variables)
    G_grad = G_tape.gradient(C_fake_loss, generator.trainable_variables)

    C_opt.apply_gradients(zip(C_grad, classifier.trainable_variables))
    G_opt.apply_gradients(zip(G_grad, generator.trainable_variables))

    return C_real_loss, C_fake_loss

def train(dataset, epochs):
    pbar = tqdm(range(epochs))
    losses = []
    for epoch in pbar:
        for batch in dataset:
            #print(batch[0].shape, batch[1].shape)
            _loss = train_step(x=batch[0], y=batch[1])
            losses.append(_loss)
        pbar.set_description(
                '[Joint Training of Classifier & Generator] | C-CrossEnt: {:.4f} | G-Ent: {:.4f}'.format(
                        *np.array(losses).mean(0)))

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 128
TEST_BATCH_SIZE = 1024
z_dim = 64

(train_images, train_classes), (test_images, test_classes) = load_data()
train_images, test_images = train_images.astype(np.float32)/255, test_images.astype(np.float32)/255
train_images, test_images = train_images*2 - 1, test_images*2 - 1
train_images, test_images = np.expand_dims(train_images, 3), np.expand_dims(test_images, 3)

enc = OneHotEncoder(sparse=False)
train_labels = enc.fit_transform(train_classes.reshape(-1,1))
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE, 0, True).batch(BATCH_SIZE)

generator = define_G(z_dim, True, False)
classifier = define_C((28, 28, 1), 10, True, False)

C_opt = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.5)
G_opt = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.5)

In [ ]:
train(train_dataset, 2)

In [ ]:
generator.save_weights("generator_kl_epoch_{}.h5".format(100))
classifier.save_weights("classifier_kl_epoch_{}.h5".format(100))

In [ ]:
(c_train_images, c_train_classes), (c_test_images, c_test_classes) = tf.keras.datasets.cifar10.load_data()
c_train_images, c_test_images = c_train_images.astype(np.float32)/255, c_test_images.astype(np.float32)/255
c_train_images, c_test_images = c_train_images*2 - 1, c_test_images*2 - 1

In [ ]:
resized_cifar10 = []
for img in c_test_images:
    gray_img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
    resized_cifar10.append(cv2.resize(gray_img, (28, 28), interpolation=cv2.INTER_CUBIC))
cifar10_images = np.array(resized_cifar10)
cifar10_images = np.expand_dims(cifar10_images, 3)

In [ ]:
# generator.load_weights("generator_kl_epoch_100.h5")
# classifier.load_weights("classifier_kl_epoch_100.h5")
generator.load_weights("generator_epoch_500.h5")
classifier.load_weights("classifier_epoch_500.h5")

In [ ]:
z = np.random.uniform(-1,1,(10000, 64))
generated_images = generator.predict(z)

In [ ]:
rows, columns = 10, 10
figure, axes = plt.subplots(rows, columns)
for row in range(rows):
    for col in range(columns):
        image = generated_images[row + rows * col, :, :, :]
        axes[row, col].imshow(np.squeeze(image), cmap='gray')
        axes[row, col].axis('off')
figure.savefig('C:/GAN/venv/images/{}.png'.format('test1'))
plt.close()

In [ ]:
test_result1 = classifier.predict(generated_images)
test_result2 = classifier.predict(cifar10_images)
test_result3 = classifier.predict(test_images)

In [ ]:
predict_labels = np.argmax(test_result3,1)
accuracy_score(test_classes,predict_labels)

In [ ]:
arr = np.ones(10000)
len(arr[np.max(test_result2,1) < 0.99999])

In [ ]:
def define_Conditional_G(z_dim, n_classes=10, use_bias=False, print_summary=True):
    kernel_init = 'glorot_normal'
    
    in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
    z_in = Input(shape=(z_dim,))

    net = Dense(units=128, kernel_regularizer=l2(1E-5), use_bias=use_bias)(z_in)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)
   
    net = Dense(units=7*7*32, kernel_regularizer=l2(1E-5), use_bias=use_bias)(net)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)
    net = Reshape((7, 7, 32))(net)
    
    merge = Concatenate()([net,li])
    
    net = Conv2DTranspose(filters=16, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                          kernel_regularizer=l2(1E-5), padding='same', kernel_initializer=kernel_init)(merge)
    net = BatchNormalization()(net)
    net = LeakyReLU(alpha=0.15)(net)

    out = Conv2DTranspose(filters=1, kernel_size=(5, 5), strides=(2, 2), use_bias=use_bias,
                          padding='same', kernel_initializer=kernel_init, activation='tanh')(net)

    model = tf.keras.models.Model(inputs=[z_in,in_label], outputs=out, name='Generator')

    if print_summary:
        print(model.summary())

    return model